In [3]:
import os
import torch
import transformers
from datasets import load_dataset

%load_ext autoreload
%autoreload 2
from datasets import Dataset
import sys
sys.path.insert(1, '../stage2')
from labels_to_nn import LabelToNN

In [1]:
import torch

In [2]:
raw_datasets = load_dataset("conll2003")

Found cached dataset conll2003 (/home/kostia/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
raw_datasets['train']

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [4]:
raw_datasets['train']['ner_tags'][:3]

[[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2], [5, 0]]

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [10]:
dir_name = '../dataset/HF_dataset/'
files = os.listdir(dir_name)
files

['final.json', '12.json', '11.json']

In [11]:
result = {'tokens': [],
          'ner_tags': []}
for file in files:
    path = dir_name + file
    translater = LabelToNN()
    translater.load(path)
    tmp = translater.run()
    result['tokens'].extend(tmp['tokens'])
    result['ner_tags'].extend(tmp['ner_tags'])

In [12]:
len(result['ner_tags'])

221

In [13]:
raw_dataset = Dataset.from_dict(result)
raw_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 221
})

In [20]:
raw_dataset.push_to_hub("kosta-naumenko/medflex")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [4]:
raw_dataset = load_dataset("kosta-naumenko/medflex")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/221 [00:00<?, ? examples/s]

In [9]:
len(raw_dataset['train'])

221

In [22]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 221
    })
})

In [19]:
raw_dataset['tokens'][0][:10], raw_dataset['ner_tags'][0][:10]

(['Отмечает',
  'постепенный',
  'набор',
  'массы',
  'тела',
  'с',
  '30',
  'лет,',
  'в',
  'настоящее'],
 [0, 1, 2, 2, 2, 0, 0, 0, 0, 0])

In [22]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "viktoroo/sberbank-rubert-base-collection3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

In [41]:
inputs = tokenizer(raw_dataset[0]["tokens"], is_split_into_words=True)
inputs.tokens()[:10]

['[CLS]', 'рост', '170', 'см', '.', 'вес', '100', ',', '0', 'кг']

In [42]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [43]:
labels = raw_dataset[0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 1, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [44]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [45]:
tokenized_datasets = raw_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_dataset.column_names,
)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [60]:
input_ids, _, _, labels = list(tokenized_datasets[0].values())

In [63]:
for i in range(len(input_ids)):
    if labels[i] > 0:
        if labels[i] == 1:
            print(" ")
        print(tokenizer.decode(input_ids[i]), end='')
        

 
питание:повышенное. 
ад,ммрт.ст.:160/100. 
отек##и:пасто##зно##стьстопиголе##не##и. 
усиленлего##чны##ирисуноквн/о. 
сердцерасширен##овлево. 
п##б##п##н##пг,

In [66]:
raw_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 20
})

In [54]:
list(tokenized_datasets[0].keys())

['input_ids', 'token_type_ids', 'attention_mask', 'labels']